In [1]:
import pickle as pkl 

In [2]:
import re
import pandas as pd
import json
import ast

def extract_markdown_blocks(text):
    """
    Finds all content inside triple-backticks (``` ... ```).
    Returns a list of string blocks (including JSON blocks).
    """
    pattern = r'```(.*?)```'
    blocks = re.findall(pattern, text, flags=re.DOTALL)
    return blocks

def parse_table_block(table_text):
    """
    Parse a single Markdown table block into a pandas DataFrame.
    Assumes the block is a valid Markdown table (pipe-delimited).
    Skips the '---' separator line.
    """
    lines = table_text.strip().split('\n')
    # Remove the potential separator line (e.g. |---|---|...|), typically lines[1].
    if len(lines) > 2:
        filtered_lines = [lines[0]] + lines[2:]
    else:
        filtered_lines = lines  # fallback if there's no clear separator line

    table_rows = []
    for line in filtered_lines:
        line = line.strip()
        if not line:
            continue  # skip empty lines
        cells = [cell.strip() for cell in line.strip('|').split('|')]
        table_rows.append(cells)

    if not table_rows:
        return pd.DataFrame()

    header = table_rows[0]
    data = table_rows[1:]
    df = pd.DataFrame(data, columns=header)
    return df

def parse_combined_block(block):
    """
    When you have a single code block that contains multiple tables separated by a blank line,
    followed by a JSON/Python dict, this function:
      1) Splits by double newlines ('\\n\\n').
      2) Parses the first N-1 blocks as tables, and the last block as JSON/Python dict.
    
    Returns a list of DataFrames (tables) and the final dict (JSON).
    """
    # Split by double newlines
    parts = block.strip().split('\n\n')
    if len(parts) < 2:
        # Not enough parts to have both table(s) and JSON, so just try to parse as a single table
        return [parse_table_block(block)], {}

    tables = []
    # Parse all but the last as tables
    for table_part in parts[:-1]:
        tables.append(parse_table_block(table_part))

    # The last part is JSON/Python dict
    last_part = parts[-1].strip()
    # We can try to parse with literal_eval (if it uses single quotes)
    # or fall back to json.loads if it looks like standard JSON with double quotes.
    # We'll do a quick heuristic:
    if '"' in last_part or last_part.strip().startswith('{'):
        # Attempt JSON
        try:
            json_data = json.loads(last_part)
        except json.JSONDecodeError:
            # fallback to ast.literal_eval
            json_data = ast.literal_eval(last_part)
    else:
        # Likely single quotes or Python literal
        json_data = ast.literal_eval(last_part)

    return tables, json_data

def parse_tables_and_json(text):
    """
    Master function that handles both:
      - The "original" scenario of FOUR triple-backtick blocks (3 tables + 1 JSON).
      - The "combined" scenario of ONE triple-backtick block containing multiple tables + JSON.

    If more than one triple-backtick block is found:
        * If exactly four: parse them as table1, table2, table3, JSON.
        * Otherwise, you could extend logic to parse more blocks as needed.
    If exactly one triple-backtick block is found:
        * Parse it via parse_combined_block (tables separated by blank lines, plus final JSON).
    If no triple-backtick blocks are found, you can decide how to handle it (return empty?).
    """
    blocks = extract_markdown_blocks(text)

    if len(blocks) == 0:
        # No triple-backtick blocks found, so just return empty or handle differently
        return [], {}

    elif len(blocks) == 1:
        # We have a single code block that might contain multiple tables + JSON
        table_list, json_data = parse_combined_block(blocks[0])
        return table_list, json_data

    elif len(blocks) >= 4:
        # Original scenario: parse the first 3 as tables, 4th as JSON
        # (If you actually have more than 4 blocks, you can decide how to handle them.)
        table1 = parse_table_block(blocks[0])
        table2 = parse_table_block(blocks[1])
        table3 = parse_table_block(blocks[2])
        # parse 4th block as JSON
        # If it uses a label "JSON" inside the block, remove that. Or just do a try/except.
        block4 = blocks[3].replace('JSON', '')
        json_data = json.loads(block4)
        return [table1, table2, table3], json_data

    else:
        # We have 2 or 3 triple-backtick blocks, not matching the original expectation of 4.
        # You can adapt logic here. As an example, parse them all as tables except the last as JSON.
        table_list = []
        # parse all but the last
        for block in blocks[:-1]:
            table_list.append(parse_table_block(block))
        # parse last as JSON
        last_block = blocks[-1].replace('JSON', '')
        try:
            json_data = json.loads(last_block)
        except json.JSONDecodeError:
            # fallback for single-quote or python-literal style
            json_data = ast.literal_eval(last_block)
        return table_list, json_data


In [3]:
import json
import re
from typing import List, Dict, Any

def parse_result(result: str) -> Dict[str, Any]:
    # Split sections using headers
    sections = re.split(r'\*\*([^\*]+)\*\*\s*\n', result)
    parsed = {
        'row_column_stats': [],
        'detailed_column_stats': [],
        'detailed_cell_stats': [],
        'cell_level_diff': {}
    }
    
    current_table = None
    current_json = None
    
    for i, section in enumerate(sections):
        section = section.strip()
        if not section:
            continue
            
        # Detect table headers
        if section == 'Row and Column Stats':
            current_table = 'row_column_stats'
        elif section == 'Detailed Column Stats':
            current_table = 'detailed_column_stats'
        elif section == 'Detailed Cell Stats':
            current_table = 'detailed_cell_stats'
        elif section == 'Cell Level Difference with Magnitude':
            current_json = True
            
        # Process tables
        if current_table and ('|' in section or '`' in section):
            table_data = []
            in_table = False
            
            for line in section.split('\n'):
                line = line.strip().strip('`')
                if not line:
                    continue
                    
                if line.startswith('|') and '---' not in line:
                    in_table = True
                    parts = [p.strip() for p in line.split('|') if p.strip()]
                    if not table_data:  # Header row
                        headers = parts
                    else:
                        table_data.append(dict(zip(headers, parts)))
                elif in_table:
                    break  # End of table
                    
            if table_data:
                parsed[current_table] = table_data
                current_table = None
                
        # Process JSON
        elif current_json and section.startswith('```JSON'):
            json_str = re.search(r'```JSON\n(.*?)\n```', section, re.DOTALL)
            if json_str:
                try:
                    parsed['cell_level_diff'] = json.loads(json_str.group(1))
                except json.JSONDecodeError:
                    parsed['cell_level_diff'] = {}
            current_json = False
            
    return parsed

def extract_results(data: List[Dict]) -> List[Dict]:
    results = []
    for item in data:
        if 'result' in item:
            parsed = parse_result(item['result'])
            results.append(parsed)
            print(parsed)
    return results

# Usage example:


In [4]:
import pickle as pkl

In [5]:
with open("/Users/turning/Desktop/TANQ/eval_dataset/human_annotation/full_dataset/full_dataset_gpt_stats_with_score_new.pkl", "rb") as f:
    gpt_statistics = pkl.load(f)

In [6]:
len(gpt_statistics)

250

In [7]:
    # notes += "STRING_TUPLE: (column name, ner_mismatch, spell_errors, abbreviated_string, semantically_same, semantically_different, other)\n"
    # notes += "NUMERICAL_TUPLE: (column name, unit_mismatch, ner_mismatch, delta)\n"
    # notes += "BOOLEAN_TUPLE: (same, different)\n"
    # notes += "LIST_TUPLE: (MI, EI, EM)\n\n"
    # md_table = """Type| Count |Differences|\n"""

In [8]:
import pandas as pd

penalties = {"cell":1 , "row":1 , "col":1, "MI": 1, "EI": 1, "Partial": 1}
datatype_penalties = {"numerical":1, "bool":1,"string": 1, "date":1, "time":1, "list":1, "others":1}


def calculate_score(table):
    pass

In [9]:
def table_to_dict_list_comparison(table_string, suffix=""):
    # If the string contains <think> and </think>, only process content after </think>
    if "</think>" in table_string:
        table_string = table_string.split("</think>", 1)[1]
    
    table_string = table_string.replace("markdown", "")
    # Split the table into lines
    lines = table_string.strip().split('\n')

    # Find the first line that contains the table header (i.e., a line with '|' and a subsequent line with '--')
    try:
        table_start_idx = next(
            (i for i, line in enumerate(lines) if '|' in line), None
        )
    except Exception as e:
        print("Error in table")
        print(table_string)

    # If no table is found, return an empty list
    if table_start_idx is None:
        return []

    # Process the header from the detected table start line
    header = lines[table_start_idx].strip().split('|')
    header = [col.strip() for col in header if col.strip()]

    # Prepare the list to hold dictionaries
    table_as_dicts = []

    # Loop through each data row, skipping any separator rows and stopping at ``` or blank lines
    for line in lines[table_start_idx + 1:]:
        # Stop processing if the table ends
        if '```' in line or not line.strip():
            break

        # Skip lines that contain only '---'
        if '---' in line:
            continue

        row_values = line.strip().split('|')
        row_values = [val.strip() for val in row_values if val.strip()]

        # Create a dictionary for the current row, ensuring to match header order with values
        row_dict = {}
        for i in range(len(header)):
            row_dict[header[i]] = row_values[i] if i < len(row_values) else None

        table_as_dicts.append(row_dict)

    return table_as_dicts


In [10]:
def _get_column_type_count(table, n_rows, type):

    if len(table) < n_rows:
        return 0
    mask = table.map(lambda x: (
        isinstance(x, tuple) and x[0] == type))
    type_count = mask.all(axis=0).sum()
    # print(type_count, "COLUMN COUNT")
    # Remove columns that match the condition
    table.drop(columns=table.columns[mask.all(axis=0)], inplace=True)
    cols_to_remove = []
    for col_header in table.columns:
        parts = col_header.split("/")
        if type == "MI" and ((len(parts) == 1 and "T1" in parts[0]) or (len(parts) == 2 and parts[1].strip() in ['-', ""])):
            cols_to_remove.append(col_header)
            type_count += 1
        elif type == "EI" and ((len(parts) == 1 and "T1" in parts[0]) or (len(parts) == 2 and parts[0].strip() in ['-', ""])):
            cols_to_remove.append(col_header)
            type_count += 1
    table.drop(columns=cols_to_remove, inplace=True)
    return type_count


def _get_row_type_count(table, n_cols, type):
    numm_cols = len(table.columns)
    if numm_cols < n_cols:
        return 0
    mask = table.map(lambda x: (
        isinstance(x, tuple) and x[0] == type))
    # print(mask)
    type_count = mask.all(axis=1).sum()
    # print(type_count, "ROW COUNT")
    # Remove rows that match the condition
    table.drop(index=table.index[mask.all(axis=1)], inplace=True)

    return type_count

In [11]:
def _get_cell_type_count(table, type):

    mask = table.map(lambda x: x == type or (
        isinstance(x, tuple) and x[0] == type))
    type_count = mask.sum().sum()
    
    # Remove cells that match the condition
    table[mask] = None
    
    return type_count

In [12]:
for i, table in enumerate(gpt_statistics):
    if "stats table" in table:
        gpt_statistics[i]['partial_cell_delta_stats'] = table['stats table']

In [13]:
def parse_partial_cell_stats_table(data):
    
    lines = data.strip().split("\n")
    Type_index = None
    for i, line in enumerate(lines):
        if "Type" in line:
            Type_index = i
            break
    lines = lines[Type_index:]
    headers = lines[0].split("|")
    parsed_data = {}
    
    for line in lines[1:]:
        parts = [part.strip() for part in line.split("|")]
        parsed_data.update({
            parts[0]: [int(parts[1]), parts[2] if len(parts) > 2 else ""]

        })
    
    return parsed_data

In [14]:
import re

def parse_nested_lists(input_str):

    list_contents = re.findall(r'\[(.*?)\]', input_str)
    result = []
    
    for content in list_contents:
        elements = []
        for elem in content.split(','):
            elem = elem.strip()
            try:
                elements.append(int(elem))
            except ValueError:
                elements.append(elem)
        result.append(elements)
    
    return result

In [15]:
from pprint import pprint

for idx, table in enumerate(gpt_statistics):
    score = 0
    # print(table['comparison_tuples'])
    if table['df_wo_em'].empty:
        gpt_statistics[idx]['table_score'] = 0
        continue
    alignment = table_to_dict_list_comparison(table["alignment"])
    n_rows = len(alignment)
    n_cols = len(alignment[0])
    ei_mi_table = pd.DataFrame(table["ei_mi_table"])
    print(ei_mi_table)

    ei_column_count = _get_column_type_count(ei_mi_table, n_rows, "EI")
    mi_column_count = _get_column_type_count(ei_mi_table, n_rows, "MI")
    ei_row_count = _get_row_type_count(ei_mi_table, n_cols, "EI")
    mi_row_count = _get_row_type_count(ei_mi_table, n_cols, "MI")
    ei_cell_count = _get_cell_type_count(ei_mi_table, "EI")
    mi_cell_count = _get_cell_type_count(ei_mi_table, "MI")
    # TODO Calculate penalties with cell col and row penalties, with MI and EI penalties: DONE
    ei_score = (((ei_column_count)/n_cols)*penalties['col']) + (((ei_row_count) /
                                                                n_rows)*penalties['row']) + (((ei_cell_count)/(n_rows*n_cols))*penalties['cell'])
    mi_score = (((mi_column_count)/n_cols)*penalties['col']) + (((mi_row_count) /
                                                              n_rows)*penalties['row']) + (((mi_cell_count)/(n_rows*n_cols))*penalties['cell'])
    score = penalties["EI"]*ei_score + penalties["MI"]*mi_score
    # print(table.keys())
    # print(table["partial_cell_delta_stats"])
    partial_cell_statistics = parse_partial_cell_stats_table(
        table["partial_cell_delta_stats"])
    num_penalties = 0
    # print(table["partial_cell_delta_stats"])
    # num_pp_count =
    # num_penalties = partial_cell_statistics['Numerical'][0]
    num_penalties += table['partial_scores']["Numerical"]
    print(table["partial_cell_delta_stats"])
    if partial_cell_statistics['Numerical'][1] != "":
        numeric_cols = parse_nested_lists(
            partial_cell_statistics['Numerical'][1])
        for col in numeric_cols:
            if isinstance(col[-2], str):
                col[-2] = 0
            num_penalties += col[-2]
            # num_penalties += 0.1*col[-3]
    
    string_penalties = partial_cell_statistics['String'][0]
    # string_penalties = 0
    if partial_cell_statistics['String'][1] != "":
    # TODO : add zero penalties for abbreivations and semantically similar
        str_cols = parse_nested_lists(partial_cell_statistics['String'][1])
        for col in str_cols:
            string_penalties -= (col[-3] + col[-4])

    # print(table['partial_scores']["Date"], idx)
    # print(partial_cell_statistics['Date'][0], idx)
    # datetime_penalties = table['partial_scores']["Date"] + \
    #     partial_cell_statistics['Time'][0]
    datetime_penalties = partial_cell_statistics['Date'][0] + \
        partial_cell_statistics['Time'][0]
    # boolean_penalties = int(
    #     partial_cell_statistics['Boolean'][1].split(",")[1])
    boolean_penalties = partial_cell_statistics['Boolean'][0]
    list_penalties = partial_cell_statistics['List'][0]
    other_penalties = table['type_counts']['Others']
    # cell penalties to be calculated
    cell_penalties = num_penalties*datatype_penalties['numerical'] + string_penalties*datatype_penalties['string'] + datetime_penalties * \
        datatype_penalties['date'] + boolean_penalties * \
        datatype_penalties['bool'] + list_penalties*datatype_penalties['list'] + \
        other_penalties*datatype_penalties['others']
    cell_penalties = cell_penalties/(n_rows*n_cols)
    # DONE: cell penalties calculated
    score += penalties["Partial"]*cell_penalties*penalties['cell']
    score = min(1, score)
    gpt_statistics[idx]['table_score'] = score

  Distance (yards).T1/Distance (meters).T2
0                                     None
1                                     None
2                                     None
3                                     None
4                                     None
Type| Count |Differences|
Numerical | 5 |[Distance (yards).T1/Distance (meters).T2, 5, 0, 0.008685121107266436], |
String | 0 ||
Boolean | 0 |0, 0|
Date | 0 |0|
Time | 0 |0|
List | 0 |0, 0, 0|

  Distance (yards).T1/Distance (meters).T2 Date.T1/Date.T2
0                                     None            None
1                                     None            None
2                                     None            None
3                                     None            None
4                                     None            None
Type| Count |Differences|
Numerical | 5 |[Distance (yards).T1/Distance (meters).T2, 5, 0, 0.002633217993079585], |
String | 0 ||
Boolean | 0 |0, 0|
Date | 5 |1.0|
Time | 0 |0|
List | 0 |0, 0, 0|

In [16]:
len(gpt_statistics)

250

In [17]:
gpt_statistics[0].keys()

dict_keys(['alignment', 'alignment_parsed', 'compare_table', 'df_wo_em', 'comparison_tuple', 'comparison_tuples_parsed', 'delta', 'type_counts', 'empty_cells', 'ei_mi_table', 'stats table', 'cell_statistics', 'row_col_statistics', 'ei_mi_column_types', 'partial_cell_delta_stats', 'table_score', 'partial_scores', 'cell_stats'])

In [18]:
for table in gpt_statistics:
    print(table.keys())
    print(table['table_score'])
    print(table['alignment'])
    print(table['comparison_tuples'])
    print("\n\n")

dict_keys(['alignment', 'alignment_parsed', 'compare_table', 'df_wo_em', 'comparison_tuple', 'comparison_tuples_parsed', 'delta', 'type_counts', 'empty_cells', 'ei_mi_table', 'stats table', 'cell_statistics', 'row_col_statistics', 'ei_mi_column_types', 'partial_cell_delta_stats', 'table_score', 'partial_scores', 'cell_stats'])
0.002171280276816609
| Distance (yards).T1/Distance (meters).T2 | Greyhound.T1/Greyhound.T2 | Time.T1/Time.T2 | Date.T1/Date.T2 |
|------------------------------------------|---------------------------|----------------|----------------|
| 325/297.48                               | Lemon Clover/Lemon Clover | 17.34/17.34    | 11.10.1996/11.10.1996 |
| 525/480.21                               | Whitty Guinness/Whitty Guinness | 28.54/28.54 | 29.10.2010/29.10.2010 |
| 550/502.92                               | Whatsupjack/Whatsupjack   | 29.91/29.91    | 18.09.2009/18.09.2009 |
| 700/640.08                               | Tinas Girl/Tinas Girl     | 38.79/38.79    |

KeyError: 'comparison_tuples'

In [19]:
# with open("/home/turning/Jainit/TANQ/eval_dataset/human_annotation/full_dataset/full_dataset_gpt_stats_with_score_new.pkl", "wb") as f:
#     pkl.dump(gpt_statistics, f)

In [20]:
import json
with open("/Users/turning/Desktop/TANQ/eval_dataset/llm_table_alignments_data/decomp_fuzzy_gpt4o.json", "r") as f:
    gpt4o_data = json.load(f)

In [21]:
with open("/Users/turning/Desktop/TANQ/eval_dataset/tables.json", "r") as f:
    tables_data = json.load(f)
    

In [22]:
our_scores = {}
for table in gpt_statistics:
    if 'table1' in table: 
        our_scores.setdefault(table['table1'], {})
        for ogtable in tables_data: 
            if ogtable['original'] == table['table1']:
                print(ogtable.keys())
                for i in range(5):
                    if ogtable[f'pertubation{i}'] == table['table2']:
                        our_scores[table['table1']]['perturbation'+str(i)] = [table['table_score'], ogtable[f'pertubation{i}']]
    else:
        for ogtable in gpt4o_data:
            for i in range(5):
                if f'alignment{i}' in ogtable:
                    if ogtable[f'alignment{i}'] == table['alignment']:
                        our_scores.setdefault(ogtable['original'], {})
                        our_scores[ogtable['original']]['perturbation'+str(i)] = [table['table_score'], ogtable['pertubation'+str(i)]]

dict_keys(['original', 'pertubation0', 'pertubation1', 'pertubation2', 'pertubation3', 'pertubation4'])
dict_keys(['original', 'pertubation0', 'pertubation1', 'pertubation2', 'pertubation3', 'pertubation4'])
dict_keys(['original', 'pertubation0', 'pertubation1', 'pertubation2', 'pertubation3', 'pertubation4'])
dict_keys(['original', 'pertubation0', 'pertubation1', 'pertubation2', 'pertubation3', 'pertubation4'])
dict_keys(['original', 'pertubation0', 'pertubation1', 'pertubation2', 'pertubation3', 'pertubation4'])
dict_keys(['original', 'pertubation0', 'pertubation1', 'pertubation2', 'pertubation3', 'pertubation4'])
dict_keys(['original', 'pertubation0', 'pertubation1', 'pertubation2', 'pertubation3', 'pertubation4'])


In [23]:
our_new_scores = []
for key, value in our_scores.items():
    new_dict = {}
    new_dict['original'] = key
    for i in range(5):
        new_dict['perturbation'+str(i)] = value[f'perturbation{i}'][1]
        new_dict[f'result{i}'] = 1- value[f'perturbation{i}'][0]
    
    our_new_scores.append(new_dict)

In [24]:
with open("/Users/turning/Desktop/TANQ/qualitative_examples/tanq_normal_weights.json", "w") as f:
    json.dump(our_new_scores, f)